In [102]:
import cv2
import numpy as np
def replaceZeroes(data):
    min_nonzero = min(data[np.nonzero(data)])
    data[data == 0] = min_nonzero
    return data
def MSR(img, scales):
    weight = 1 / 3.0
    scales_size = len(scales)
    h, w = img.shape[:2]
    log_R = np.zeros((h, w), dtype=np.float32)

    for i in range(scales_size):
        img = replaceZeroes(img)
        L_blur = cv2.GaussianBlur(img, (scales[i], scales[i]), 0)
        L_blur = replaceZeroes(L_blur)
        dst_Img = cv2.log(img/255.0)
        dst_Lblur = cv2.log(L_blur/255.0)
        dst_Ixl = cv2.multiply(dst_Img, dst_Lblur)
        log_R += weight * cv2.subtract(dst_Img, dst_Ixl)

    dst_R = cv2.normalize(log_R,None, 0, 255, cv2.NORM_MINMAX)
    log_uint8 = cv2.convertScaleAbs(dst_R)
    return log_uint8

In [103]:
if __name__ == '__main__':
    img = 'E:\\Download\\high\\our485\\high\\0.jpg'
    #img = 'E:\\Download\\high\\our485\\low\\0.jpg'
    scales = [15,101,301]  # [3,5,9]  #看不出效果有什么差别
    src_img = cv2.imread(img)
   #print(src_img)
    b_gray, g_gray, r_gray = cv2.split(src_img)
    b_gray = MSR(b_gray, scales)
    g_gray = MSR(g_gray, scales)
    r_gray = MSR(r_gray, scales)
    result = cv2.merge([b_gray, g_gray, r_gray])

    cv2.imshow('img',src_img)
    cv2.imshow('MSR_result',result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [2]:
import numpy as np
import cv2


def replaceZeroes(data):
    min_nonzero = min(data[np.nonzero(data)])
    data[data == 0] = min_nonzero
    return data

def SSR(src_img, size):
    L_blur = cv2.GaussianBlur(src_img, (size, size), 0)
    img = replaceZeroes(src_img)
    L_blur = replaceZeroes(L_blur)

    dst_Img = cv2.log(img/255.0)
    print(dst_Img)
    dst_Lblur = cv2.log(L_blur/255.0)
    dst_IxL = cv2.multiply(dst_Img,dst_Lblur)
    #print(dst_IxL)
    log_R = cv2.subtract(dst_Img, dst_IxL)
    log_R = np.exp(log_R)
    #print(log_R)
    #dst_R = cv2.normalize(log_R,None,0,255,cv2.NORM_MINMAX)
    #log_uint8 = cv2.convertScaleAbs(dst_R)
    #return log_uint8//1
    return log_R


if __name__ == '__main__':
    img = 'E:\\Download\\train_image\\our485\\high\\1.jpg'
    size = 3
    src_img = cv2.imread(img)
    b_gray, g_gray, r_gray = cv2.split(src_img)
    b_gray = SSR(b_gray, size)
    g_gray = SSR(g_gray, size)
    r_gray = SSR(r_gray, size)
    result = cv2.merge([b_gray, g_gray, r_gray])
    print(result)
    cv2.imshow('img',src_img)
    cv2.imshow('result',result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

[[-1.15923691 -1.17181569 -1.15923691 ... -0.61400986 -0.61400986
  -0.61400986]
 [-1.17181569 -1.17181569 -1.17181569 ... -0.60678961 -0.60678961
  -0.60678961]
 [-1.19745812 -1.15923691 -1.17181569 ... -0.60678961 -0.60678961
  -0.61400986]
 ...
 [-1.27858367 -1.27858367 -1.2508041  ... -0.35947999 -0.35947999
  -0.35387774]
 [-1.27858367 -1.2927683  -1.2508041  ... -0.36511381 -0.34276651
  -0.35387774]
 [-1.2927683  -1.2508041  -1.27858367 ... -0.36511381 -0.37077955
  -0.35947999]]
[[-1.1345443  -1.1345443  -1.1345443  ... -0.65846162 -0.65846162
  -0.65091442]
 [-1.1345443  -1.17181569 -1.15923691 ... -0.65091442 -0.65091442
  -0.65846162]
 [-1.19745812 -1.17181569 -1.1345443  ... -0.65091442 -0.65846162
  -0.65091442]
 ...
 [-1.27858367 -1.2927683  -1.27858367 ... -0.44751334 -0.43531807
  -0.43531807]
 [-1.2927683  -1.27858367 -1.22377543 ... -0.44751334 -0.44751334
  -0.43531807]
 [-1.2927683  -1.2508041  -1.2927683  ... -0.44751334 -0.44751334
  -0.44751334]]
[[-1.08691625 -1

In [58]:
print(np.log(2.71828))

0.999999327347282


In [8]:
import numpy as np
import cv2


def replaceZeroes(data):
    min_nonzero = min(data[np.nonzero(data)])
    data[data == 0] = min_nonzero
    return data

def simple_color_balance(input_img, s1, s2):
    h, w = input_img.shape[:2]
    temp_img = input_img.copy()
    one_dim_array = temp_img.flatten()
    sort_array = sorted(one_dim_array)

    per1 = int((h * w) * s1 / 100)
    minvalue = sort_array[per1]

    per2 = int((h * w) * s2 / 100)
    maxvalue = sort_array[(h * w) - 1 - per2]

    # 实施简单白平衡算法
    if (maxvalue <= minvalue):
        out_img = np.full(input_img.shape, maxvalue)
    else:
        scale = 255.0 / (maxvalue - minvalue)
        out_img = np.where(temp_img < minvalue, 0,temp_img)   # 防止像素溢出
        out_img = np.where(out_img > maxvalue, 255,out_img)   # 防止像素溢出
        out_img = scale * (out_img - minvalue)                # 映射中间段的图像像素
        out_img = cv2.convertScaleAbs(out_img)
    return out_img


def MSRCP(img, scales, s1, s2):
    h, w = img.shape[:2]
    scales_size = len(scales)
    B_chan = img[:, :, 0]
    G_chan = img[:, :, 1]
    R_chan = img[:, :, 2]
    log_R = np.zeros((h, w), dtype=np.float32)
    array_255 = np.full((h, w),255.0,dtype=np.float32)

    I_array = (B_chan + G_chan + R_chan) / 3.0
    I_array = replaceZeroes(I_array)

    for i in range(0, scales_size):
        L_blur = cv2.GaussianBlur(I_array, (scales[i], scales[i]), 0)
        L_blur = replaceZeroes(L_blur)
        dst_I = cv2.log(I_array/255.0)
        dst_Lblur = cv2.log(L_blur/255.0)
        dst_ixl = cv2.multiply(dst_I, dst_Lblur)
        log_R += cv2.subtract(dst_I, dst_ixl)
    MSR = log_R / 3.0
    Int1 = simple_color_balance(MSR, s1, s2)

    B_array = np.maximum(B_chan,G_chan,R_chan)
    A = np.minimum(array_255 / B_array, Int1/I_array)
    R_channel_out = A * R_chan
    G_channel_out = A * G_chan
    B_channel_out = A * B_chan

    MSRCP_Out_img = cv2.merge([B_channel_out, G_channel_out, R_channel_out])
    MSRCP_Out = cv2.convertScaleAbs(MSRCP_Out_img)

    return MSRCP_Out

if __name__ == '__main__':
   # img = 'E\\me.jpg'
    img = 'E:\\Download\\high\\our485\\low\\12.jpg'
    scales = [15,101,301]
    s1, s2 = 2,3
    src_img = cv2.imread(img)
    result = MSRCP(src_img, scales, s1, s2)

    cv2.imshow('img',src_img)
    cv2.imshow('MSR_result',result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


D:\Anaconda\envs\pytorch_gpu\lib\site-packages\ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in true_divide


In [6]:
img = 'E:\\Download\\high\\our485\\low\\12.jpg'
src_img = cv2.imread(img)
cv2.imshow('img',src_img)
#cv2.imshow('MSR_result',result)
cv2.waitKey(0)
cv2.destroyAllWindows()